In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import time

from openai import OpenAI


class Sora2:
    def __init__(self):
        self.openai = OpenAI()

    def generate_video(self, prompt: str, file_name: str, seconds="12", size="720x1280", model="sora-2"):
        if ".mp4" not in file_name:
            file_name = file_name + ".mp4"

        video = self.openai.videos.create(
            model=model,
            prompt=prompt,
            # input_reference=None,
            seconds=seconds,
            size=size
        )

        print("Video generation started:\n", video)

        progress = getattr(video, "progress", 0)
        bar_length = 30

        while video.status in ("in_progress", "queued"):
            # Refresh status
            video = self.openai.videos.retrieve(video.id)
            progress = getattr(video, "progress", 0)

            filled_length = int((progress / 100) * bar_length)
            bar = "=" * filled_length + "-" * (bar_length - filled_length)
            status_text = "Queued" if video.status == "queued" else "Processing"

            print(f"{status_text}: [{bar}] {progress:.1f}%", end="\r", flush=True)
            time.sleep(1)

        # Move to next line after progress loop
        print("\n")

        if video.status == "failed":
            message = getattr(
                getattr(video, "error", None), "message", "Video generation failed"
            )
            print(message)
        else:
            print("Video generation completed:", video)
            print("Downloading video content...")

            content = self.openai.videos.download_content(video.id, variant="video")
            content.write_to_file(file_name)

            print(f"Wrote {file_name}")


In [ ]:
from enum import Enum


class SoraModel(Enum):
    SORA_2 = "sora-2"
    SORA_2_PRO = "sora-2-pro"

class SoraVideoSize(Enum):
    SORA_2_720x1280 = "720x1280"
    SORA_2_1280x720 = "1280x720"
    SORA_2_PRO_720x1280 = "720x1280"
    SORA_2_PRO_1280x720 = "1280x720"
    SORA_2_PRO_1024x1792 = "1024x1792"
    SORA_2_PRO_1792x1024 = "1792x1024"

class SoraVideoDuration(Enum):
    SECONDS_4 = "4"
    SECONDS_8 = "8"
    SECONDS_12 = "12"


class SoraDownloadVariant(Enum):
    VIDEO = "video"
    THUMBNAIL = "thumbnail"
    SPRITESHEET = "spritesheet"


PROMPT = """
Format & Look
Duration 4s; 180° shutter; digital capture emulating 65 mm photochemical contrast; fine grain; subtle halation on speculars; no gate weave.

Lenses & Filtration
32 mm / 50 mm spherical primes; Black Pro-Mist 1/4; slight CPL rotation to manage glass reflections on train windows.

Grade / Palette
Highlights: clean morning sunlight with amber lift.
Mids: balanced neutrals with slight teal cast in shadows.
Blacks: soft, neutral with mild lift for haze retention.

Lighting & Atmosphere
Natural sunlight from camera left, low angle (07:30 AM).
Bounce: 4x4 ultrabounce silver from trackside.
Negative fill from opposite wall.
Practical: sodium platform lights on dim fade.
Atmos: gentle mist; train exhaust drift through light beam.

Location & Framing
Seoul urban commuter platform, dawn.
Foreground: yellow safety line, a bottle of whiskey on bench.
Midground: waiting passengers silhouetted in haze.
Background: arriving train braking to a stop.
Avoid signage or corporate branding.

Wardrobe / Props / Extras
Main subject: mid-30s korean street male dancer, black baseball cap, navy hoodie, backpack with some key-rings, holding phone loosely at side.
Extras: commuters in muted tones; one cyclist pushing bike.
Props: paper coffee cup, rolling luggage, LED departure board (generic destinations).

Sound
Diegetic only: faint rail screech, train brakes hiss, distant announcement muffled (-20 LUFS), low ambient hum.
Footsteps and paper rustle; no score or added foley.

Optimized Shot List (2 shots / 4 s total)

0.00-2.40 — “Arrival Drift” (32 mm, shoulder-mounted slow dolly left)
Camera slides past platform signage edge; shallow focus reveals dancer mid-frame looking down tracks. Morning light blooms across lens; train headlights flare softly through mist. Purpose: establish setting and tone, hint anticipation.

2.40-4.00 — “Turn and Pause” (50 mm, slow arc in)
Cut to tighter over-shoulder arc as train halts; dancer turns slightly toward camera, catching sunlight rim across cheek and phone screen reflection. Eyes flick up toward something unseen. Purpose: create human focal moment with minimal motion.

Camera Notes (Why It Reads)
Keep eyeline low and close to lens axis for intimacy.
Allow micro flares from train glass as aesthetic texture.
Preserve subtle handheld imperfection for realism.
Do not break silhouette clarity with overexposed flare; retain skin highlight roll-off.

Finishing
Fine-grain overlay with mild chroma noise for realism; restrained halation on practicals; warm-cool LUT for morning split tone.
Mix: prioritize train and ambient detail over footstep transients.
Poster frame: traveler mid-turn, golden rim light, arriving train soft-focus in background haze.
"""

sora = Sora2()
sora.generate_video(
    prompt=PROMPT,
    file_name="sora_2_video.mp4",
    seconds="12",
    size="1024x1792",
    model="sora-2-pro"
)

Video generation started:
 Video(id='video_68f4c510426481938a8783a8d33c07a30d53eb2df05c18bd', completed_at=None, created_at=1760871696, error=None, expires_at=None, model='sora-2-pro', object='video', progress=0, remixed_from_video_id=None, seconds='12', size='1024x1792', status='queued')
Processing: [==============================] 100.0%

Video generation completed: Video(id='video_68f4c510426481938a8783a8d33c07a30d53eb2df05c18bd', completed_at=1760872524, created_at=1760871696, error=None, expires_at=1760876124, model='sora-2-pro', object='video', progress=100, remixed_from_video_id=None, seconds='12', size='1024x1792', status='completed')
Wrote video.mp4


In [ ]:
video_id = 'video_68f4c510426481938a8783a8d33c07a30d53eb2df05c18bd'
content = sora.openai.videos.download_content(video_id, variant="spritesheet")
content.write_to_file("spritesheet.png")